In [ ]:
# Importem llibreries necessaries per implementar l'Arbre de Decisió
import pandas as pd
import numpy as np
import openpyxl
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score, recall_score

In [ ]:
#Carreguem fitxer Excel i el llegim per poder ser utilitzat
xls = ''
fitxer = openpyxl.load_workbook(xls)
df = pd.read_excel(xls, sheet_name = '', engine = 'openpyxl')

In [ ]:
# Mostrem les 5 primeres files per conèixer com és el fitxer
df.head()

In [ ]:
# Mostrem forma del fitxer (files i columnes)
np.shape(df)

In [ ]:
# Agrupem per la variable Target per saber com d'equilibrat està el conjunt de dades
df.groupby('').size()

In [ ]:
#Separem les característiques, Dades estaran totes les variables excepte la de Target, a Target només ella mateixa
variables = len(df.columns) 
Dades = df.iloc[:,0:variables-1]
Target = df.iloc[:,variables-1]

In [ ]:
#Separarem les dades en una part d'entrenament i una part de test, 80% d'entrenament i 20% de test
Dades_train, Dades_test, Target_train, Target_test = train_test_split(Dades, Target, train_size = 0.8)

In [ ]:
#Entrenem el model, li passem una profunditat de l'arbre de 13, s'ha determinar previament que era la que proporcionava més precisió
model = DecisionTreeClassifier(max_depth=13)
# S'entrena el model només amb les variables d'entrenament
model.fit(Dades_train, Target_train)

In [ ]:
#Visualitzem l'arbre amb les mesures que triem
plt.figure(figsize=[80,35])
plot_tree(decision_tree=model, feature_names=Dades.columns, filled = True, fontsize=25);
# Guardem l'imatge de l'arbre a la carpeta que s'indiqui
#plt.savefig('ArbreDesicio.png')

In [ ]:
#Calculem la predicció de les dades
prediccio= model.predict(Dades_test)
print(prediccio)

In [ ]:
#Mostrem plot de la variable X en relació a la quantitat que hi ha a la base de dades
sb.histplot(x = df.X, hue = df.OBE1TY_5Y_grup)

In [ ]:
#Fem la matriu de confusió per veure la quantitat de 0 / 1 que el model és capaç d'encertar
MatriuConfusio = confusion_matrix(Target_test, prediccio)
print(MatriuConfusio)

In [ ]:
#Precisió total 
PrecisioTotal = np.sum(MatriuConfusio.diagonal()/np.sum(MatriuConfusio))
print(PrecisioTotal)

In [ ]:
# % de precisió de no patir obesitat
PrecisioNo = (MatriuConfusio[0,0])/sum(MatriuConfusio[0,])
print(PrecisioNo)

In [ ]:
# % de precisió de patir obesitat
PrecisioSi = (MatriuConfusio[1,1])/sum(MatriuConfusio[1,])
print(PrecisioSi)

In [ ]:
#Accuracy del model d'arbre de decisió
accuracy_score(Target_test, prediccio)

In [ ]:
#Report de les mètriques per comprovar el model
print(classification_report(Target_test, prediccio))

In [ ]:
# Apliquem Shapley value per tal de determinar la importancia de les característiques (varibles)
import shap 

# Escollim el de tree shap, ja que es tracta d'un arbre de decisió
value = shap.TreeExplainer(model=model)

In [ ]:
# Agafem una fila que a la varible target sigui = 0

valor_zero = Dades_train.iloc[2]
classe_zero = Target_train.iloc[2]

# Agafem una fila que a la varible target sigui = 1

valor_un = Dades_train.iloc[5]
classe_un = Target_train.iloc[5]

In [ ]:
# Predicció del cas amb Target = 0

print(f"{model.predict_proba(valor_zero.values.reshape(1,-1))}")

# En aquest cas Target = 0, té sentit que predigui 100% que és 0 i 0% que és 1

In [ ]:
# Predicció del cas amb Target = 1

print(f"{model.predict_proba(valor_un.values.reshape(1,-1))}")

# En aquest cas Target = 1, prediu malament ja que aquest model no calcula bé per els valors 1 = obesos

In [ ]:
# Mirem quines són les característiques més impotants en el model
model.feature_importances_

In [ ]:
# Calculem els valors Shapley (amb la instrucció shap_values) de cada característica pel cas de Target = 0
# Valors de Shapley per a la classe 0 (no obesitat)

shap_valor_zero = value.shap_values(valor_zero)
print(f"{shap_valor_zero[0]}") 

In [ ]:
# Valors de Shapley per a la classe 1 (obesitat)

shap_valor_zero = value.shap_values(valor_zero)
print(f"{shap_valor_zero[1]}") 

In [ ]:
# Calculem els valors Shapley (amb la instrucció shap_values) de cada característica pel cas de Target = 1
# Valors de Shapley per a la classe 0 (no obesitat) 

shap_valor_un = value.shap_values(valor_un)
print(f"{shap_valor_un[0]}")

In [ ]:
# Valors de Shapley per a la classe 1 (obesitat) 

shap_valor_un = value.shap_values(valor_un)
print(f"{shap_valor_un[1]}")

In [ ]:
# Plotejem per veure que volen dir aquests valors

shap_values = value.shap_values(Dades_train)

shap.summary_plot(shap_values[1], Dades_train)

# Les variables estan ordenades de manera que la primera variable té més importància i l'última és la que en té menys.
# El color ens indica vermell = valors alts si es així ls valors de Shapley seran baixos i ens indica que la característica empenya cap a la classe negativa,
# és a dir Target 0 (no obés), blau = valors baixos indica valors de Shapley alts i per tant Target = 1 (obés)

In [ ]:
# Veiem un gràfic més clar

shap.summary_plot(shap_values, Dades_train)